# Downselection
### Background
We are three criteria to select five boxes for this inital screen:
1. Nisin biosythesis potential
    - One of the pathogens we are screening against is Cutibacterium acnes, and it looks as though it is suseptible to [nisins](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10295553/).
    - Nisins are a subclass of lanthipeptides, so we may also want to expand our search by looking for lanthipeptides as well
2. Coagulase-negative Staphlococci (CoNS)
    - There is abundant literature showing that CoNS compete with pathogenic Staphylococcus aureus (one [example](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5117632/)).
    - We have 14 different CoNS species, as defined by this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4187637/).
3. Diversity
    - As always, we are interested in diversity.
    - I used both genus and species richness.

We select at the box level due to the time consuming practice of cherry picking multiple boxes from the freezer.

## Setup

In [1]:
import pandas as pd
import pyarrow.parquet as pq
import boto3
import os
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor
ATHENA_RESULTS = os.environ["STAGING_BUCKET"]
ATHENA_CURSOR = connect(s3_staging_dir=ATHENA_RESULTS,
                        region_name='us-west-2',
                        cursor_class=PandasCursor,
                        profile_name="prod").cursor()
athena = boto3.client('athena')
pd.set_option("display.max_columns", 40)

## Pull AIMs with nisin BGCs and tally at box level

In [2]:
# Get bigslice output
#!aws s3 cp s3://agbiome-bigslice-output/parquet/ ../data/bigslice --recursive
# I already downloaded this

In [3]:
gcf_data = pd.read_parquet('../data/bigslice/')

In [4]:
nisins = ['BGC0000535', 'BGC0000536', 'BGC0000537', 'BGC0000538', 'BGC0001289', 'BGC0001701', 'BGC0002323']
gcf_data[gcf_data['bgc_id'].isin(nisins)]

,gcf_id,bgc_id,membership_value,rank,on_contig_edge,length_nt,bgc_source
948235,1295,BGC0000538,403,0,1,7610,MI-BiG
950650,1295,BGC0002323,312,0,1,9758,MI-BiG
952769,544,BGC0001289,313,0,1,10643,MI-BiG
960863,1295,BGC0000536,289,0,1,13530,MI-BiG
964726,1295,BGC0000535,336,0,1,15016,MI-BiG
965320,1295,BGC0000537,333,0,1,15653,MI-BiG
965697,1295,BGC0001701,293,0,1,16220,MI-BiG


In [5]:
nisin_bgcs = gcf_data[
    (gcf_data['gcf_id'] == 1295) & 
    (gcf_data['bgc_source'] == 'AgBiome')]

In [6]:
print('we have', nisin_bgcs.bgc_id.str[:9].nunique(), 'nisin-producing AIMs')

we have 229 nisin-producing AIMs


In [8]:
nisin_aims = nisin_bgcs.bgc_id.str[:9].unique()
stats = f"""
    SELECT
        p.aim,
        p.aim_number,
        p.physical_box_number
    FROM genomics.physical_box_mapping p
        LEFT JOIN restricted_use.restricted_microbe_metadata r ON p.aim = r.aim
    WHERE aim_number IN {tuple(nisin_aims)}
        AND (r.is_screenable IS NULL OR r.is_screenable = true)
    """

nisin_box = (
    ATHENA_CURSOR\
    .execute(stats)\
    .as_pandas()\
    .groupby('physical_box_number')\
    .size()\
    .reset_index()\
    .rename(columns={0: 'n_nisin'})\
    .sort_values('n_nisin', ascending = False)
)
nisin_box

,physical_box_number,n_nisin
27,263,9
130,827,5
117,786,5
99,614,4
54,432,4
...,...,...
52,375,1
51,371,1
50,366,1
47,354,1


In [9]:
stats = f"""
    SELECT
        phylum,
        genus,
        binomial,
        aim
    FROM genomics.gtdbtk_parquet
    WHERE aim IN {tuple(nisin_bgcs.bgc_id.str[3:9].astype(int))}
    """
nisin_tax = (
    ATHENA_CURSOR\
    .execute(stats)\
    .as_pandas()
)

In [10]:
nisin_tax.groupby('genus').size()

genus
Bacillus            1
Bacillus_A        148
Curtobacterium      1
Lactococcus         4
Priestia           12
Rhodococcus         1
Streptomyces        2
dtype: int64

In [11]:
nisin_tax.groupby('binomial').size()

binomial
Bacillus spizizenii                 1
Bacillus_A bombysepticus            1
Bacillus_A cereus                  17
Bacillus_A cereus_AU                1
Bacillus_A hominis                  1
Bacillus_A mycoides                 2
Bacillus_A sp002584985             14
Bacillus_A thuringiensis           43
Bacillus_A thuringiensis_AA         1
Bacillus_A thuringiensis_S         39
Bacillus_A tropicus                28
Curtobacterium flaccumfaciens_B     1
Lactococcus cremoris                1
Lactococcus lactis                  3
Priestia megaterium                12
Rhodococcus qingshengii             1
Streptomyces abikoensis             1
dtype: int64

## Pull AIMs with lanthipeptide BGCs and tally at box level

In [12]:
stats = f"""
    SELECT
        p.aim,
        p.aim_number,
        p.physical_box_number
    FROM genomics.physical_box_mapping p
        LEFT JOIN restricted_use.restricted_microbe_metadata r ON p.aim = r.aim
        RIGHT JOIN genomics.antismash a ON p.aim = a.aim
    WHERE a.product_class LIKE '%anthipeptide%'
        AND (r.is_screenable IS NULL OR r.is_screenable = true)
    """
lanthipeptide_box = (
    ATHENA_CURSOR\
    .execute(stats)\
    .as_pandas()\
    [['physical_box_number', 'aim_number']] \
    .drop_duplicates() \
    .groupby('physical_box_number')\
    .size()\
    .reset_index()\
    .rename(columns={0: 'n_lanthipeptides'})\
    .sort_values('n_lanthipeptides', ascending = False)
)
lanthipeptide_box.head(10)

,physical_box_number,n_lanthipeptides
395,451,81
399,457,81
396,452,80
141,179,79
739,846,78
788,895,75
731,838,75
755,862,75
725,832,74
733,840,74


## Pull CoNS AIMs and tally total and species richness at box level

In [13]:
CoNS= ['Staphylococcus succinus', 'Staphylococcus saprophyticus', 'Staphylococcus epidermidis',
       'Staphylococcus xylosus', 'Staphylococcus warneri', 'Staphylococcus shinii', 'Staphylococcus xylosus_B',
       'Staphylococcus pasteuri', 'Staphylococcus_A sciuri', 'Staphylococcus hominis', 'Staphylococcus equorum_A',
       'Staphylococcus haemolyticus', 'Staphylococcus capitis']
stats = f"""
    SELECT
        p.aim,
        p.aim_number,
        p.physical_box_number,
        g.binomial
    FROM genomics.physical_box_mapping p
        LEFT JOIN restricted_use.restricted_microbe_metadata r ON p.aim = r.aim
        LEFT JOIN genomics.gtdbtk_parquet g ON p.aim = g.aim
    WHERE g.binomial IN {tuple(CoNS)}
        AND (r.is_screenable IS NULL OR r.is_screenable = true)
    """
cons_box = (
    ATHENA_CURSOR\
    .execute(stats)\
    .as_pandas()
)

In [14]:
cons_div_box = (
    cons_box\
    [['physical_box_number', 'binomial']]\
    .drop_duplicates()\
    .groupby('physical_box_number')
    .size() \
    .reset_index()\
    .rename(columns={0: 'rich_cons'})\
    .sort_values('rich_cons', ascending = False)
)
cons_div_box.head(5)

,physical_box_number,rich_cons
25,797,3
24,796,3
7,369,3
3,74,2
22,774,2


In [15]:
cons_sum_box = (
    cons_box\
    [['physical_box_number', 'aim_number']] \
    .drop_duplicates() \
    .groupby('physical_box_number')\
    .size()\
    .reset_index()\
    .rename(columns={0: 'n_cons'})\
    .sort_values('n_cons', ascending = False)
)
cons_sum_box.head(5)

,physical_box_number,n_cons
15,428,12
24,796,9
25,797,8
2,33,4
7,369,3


## Calculate the genus and species richness for each box

In [16]:
stats = f"""
    SELECT
        p.aim,
        p.aim_number,
        p.physical_box_number,
        g.genus,
        g.binomial
    FROM genomics.physical_box_mapping p
        LEFT JOIN restricted_use.restricted_microbe_metadata r ON p.aim = r.aim
        LEFT JOIN genomics.gtdbtk_parquet g ON p.aim = g.aim
    WHERE r.is_screenable IS NULL 
        OR r.is_screenable = true
    """

In [17]:
species_rich_box2 = (
    ATHENA_CURSOR\
    .execute(stats)\
    .as_pandas()
)
species_rich_box2['combo'] = species_rich_box2['genus'] + species_rich_box2['binomial']
species_rich_box = (
    species_rich_box2\
    [['physical_box_number', 'combo']]\
    .drop_duplicates()\
    .groupby('physical_box_number')
    .size() \
    .reset_index()\
    .rename(columns={0: 'rich_species'})\
    .sort_values('rich_species', ascending = False)
)
species_rich_box

,physical_box_number,rich_species
326,352,54
186,202,52
157,173,44
176,192,44
385,412,43
...,...,...
545,578,1
662,701,1
663,702,1
546,579,1


In [18]:
genus_rich_box = (
    ATHENA_CURSOR\
    .execute(stats)\
    .as_pandas()\
    [['physical_box_number', 'genus']]\
    .drop_duplicates()\
    .groupby('physical_box_number')
    .size() \
    .reset_index()\
    .rename(columns={0: 'rich_genus'})\
    .sort_values('rich_genus', ascending = False)
)
genus_rich_box

,physical_box_number,rich_genus
157,173,39
1084,1149,38
152,168,35
1081,1146,35
1106,1171,34
...,...,...
239,263,1
230,252,1
276,302,1
257,281,1


## Merge all tables to make decision

In [82]:
all_tabs = (
    species_rich_box\
    .merge(genus_rich_box, how = 'outer', on = 'physical_box_number')\
    .merge(cons_sum_box, how = 'outer', on = 'physical_box_number')\
    .merge(cons_div_box, how = 'outer', on = 'physical_box_number')\
    .merge(lanthipeptide_box, how = 'outer', on = 'physical_box_number')\
    .merge(nisin_box, how = 'outer', on = 'physical_box_number')\
    .fillna(0)
)
all_tabs

,physical_box_number,rich_species,rich_genus,n_cons,rich_cons,n_lanthipeptides,n_nisin
0,352,54,23,0.0,0.0,12.0,0.0
1,202,52,31,0.0,0.0,23.0,0.0
2,173,44,39,0.0,0.0,8.0,0.0
3,192,44,33,0.0,0.0,16.0,0.0
4,412,43,25,0.0,0.0,19.0,0.0
...,...,...,...,...,...,...,...
1174,578,1,2,0.0,0.0,11.0,0.0
1175,701,1,2,0.0,0.0,0.0,0.0
1176,702,1,2,0.0,0.0,0.0,0.0
1177,579,1,2,0.0,0.0,39.0,0.0


In [101]:
all_tabs[(all_tabs.rich_species >13)].sort_values('n_nisin', ascending = False).head(5)

,n_nisin_Order,n_lanthipeptides_Order,rich_cons_Order,n_cons_Order,rich_genus_Order,rich_species_Order,physical_box_number,rich_species,rich_genus,n_cons,rich_cons,n_lanthipeptides,n_nisin,all_order
1,1,237,842,165,656,592,827,14,5,0.0,0.0,48.0,5.0,2493
4,4,219,581,967,580,509,197,16,6,0.0,0.0,49.0,4.0,2860
7,7,288,385,1055,368,242,491,24,13,0.0,0.0,43.0,3.0,2345
8,8,561,741,266,738,557,513,14,4,0.0,0.0,24.0,3.0,2871
10,10,757,74,753,53,101,149,31,25,0.0,0.0,13.0,3.0,1748


In [103]:
all_tabs[(all_tabs.rich_species >13)].sort_values('rich_cons', ascending = False).head(5)

,n_nisin_Order,n_lanthipeptides_Order,rich_cons_Order,n_cons_Order,rich_genus_Order,rich_species_Order,physical_box_number,rich_species,rich_genus,n_cons,rich_cons,n_lanthipeptides,n_nisin,all_order
1156,1156,445,2,5,415,307,369,22,12,3.0,3.0,32.0,0.0,2330
37,37,631,0,2,197,43,797,35,19,8.0,3.0,19.0,2.0,910
1016,1016,660,1,1,302,99,796,31,15,9.0,3.0,17.0,0.0,2079
175,175,849,3,8,144,42,385,35,21,2.0,2.0,9.0,0.0,1221
185,185,873,4,11,32,49,74,35,28,2.0,2.0,8.0,0.0,1154


In [102]:
all_tabs[(all_tabs.n_nisin >1)].sort_values('rich_species', ascending = False).head(5)

,n_nisin_Order,n_lanthipeptides_Order,rich_cons_Order,n_cons_Order,rich_genus_Order,rich_species_Order,physical_box_number,rich_species,rich_genus,n_cons,rich_cons,n_lanthipeptides,n_nisin,all_order
37,37,631,0,2,197,43,797,35,19,8.0,3.0,19.0,2.0,910
35,35,1064,21,3,45,82,33,32,26,4.0,1.0,1.0,2.0,1250
10,10,757,74,753,53,101,149,31,25,0.0,0.0,13.0,3.0,1748
7,7,288,385,1055,368,242,491,24,13,0.0,0.0,43.0,3.0,2345
18,18,350,555,1029,554,322,359,22,7,0.0,0.0,39.0,2.0,2828


Boxes 827, 369, 33, 796, and 797 look good

In [85]:
all_tabs.to_csv('../data/loreal_downselectioin.csv')

## Go back and describe picked boxes

In [122]:
boxes = [827, 369, 33, 796, 797]
stats = f"""
    SELECT
        p.aim,
        p.aim_number,
        p.physical_box_number,
        g.phylum,
        g.genus,
        g.binomial
    FROM genomics.physical_box_mapping p
        LEFT JOIN restricted_use.restricted_microbe_metadata r ON p.aim = r.aim
        LEFT JOIN genomics.gtdbtk_parquet g ON p.aim = g.aim
    WHERE p.physical_box_number IN {tuple(boxes)}
    """
box_descrip = (
    ATHENA_CURSOR\
    .execute(stats)\
    .as_pandas()\
    .drop_duplicates()
)
box_descrip['combo'] = box_descrip['genus'] + box_descrip['binomial']
print('This set represents', box_descrip.phylum.nunique(), 'phyla')
print('This set represents', box_descrip.genus.nunique(), 'genera')
print('This set represents', box_descrip.combo.nunique(), 'species')
print('...of which,', box_descrip[box_descrip['binomial'].isin(CoNS)].shape[0], 'are CoNS')
print('These group of CoNS represent', box_descrip[box_descrip['binomial'].isin(CoNS)].binomial.nunique(), 'of the 13 CoNS species in our collection')

This set represents 3 phyla
This set represents 45 genera
This set represents 99 species
...of which, 24 are CoNS
These group of CoNS represent 6 of the 13 CoNS species in our collection


/tmp/ipykernel_142/3655722362.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  box_descrip['combo'] = box_descrip['genus'] + box_descrip['binomial']


In [125]:
nisin_tax.merge(box_descrip, how = 'inner', on = 'aim')

,phylum_x,genus_x,binomial_x,aim,aim_number,physical_box_number,phylum_y,genus_y,binomial_y,combo
0,NaN,NaN,NaN,76090,AIM076090,797,NaN,NaN,NaN,NaN
1,Bacillota,Bacillus_A,Bacillus_A tropicus,76158,AIM076158,797,Bacillota,Bacillus_A,Bacillus_A tropicus,Bacillus_ABacillus_A tropicus
2,Bacillota,Lactococcus,Lactococcus lactis,78994,AIM078994,827,Bacillota,Lactococcus,Lactococcus lactis,LactococcusLactococcus lactis
3,Bacillota,Lactococcus,Lactococcus lactis,78996,AIM078996,827,Bacillota,Lactococcus,Lactococcus lactis,LactococcusLactococcus lactis
4,Bacillota,Lactococcus,Lactococcus lactis,78997,AIM078997,827,Bacillota,Lactococcus,Lactococcus lactis,LactococcusLactococcus lactis
5,NaN,NaN,NaN,78998,AIM078998,827,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,79003,AIM079003,827,NaN,NaN,NaN,NaN
7,Bacillota,Priestia,Priestia megaterium,3441,AIM003441,33,Bacillota,Priestia,Priestia megaterium,PriestiaPriestia megaterium
8,Bacillota,Priestia,Priestia megaterium,3441,AIM003441,33,Bacillota,Priestia,Priestia megaterium,PriestiaPriestia megaterium


In [121]:
box_descrip.drop_duplicates()

,aim,aim_number,physical_box_number,phylum,genus,binomial,combo
0,76154,AIM076154,797,NaN,NaN,NaN,NaN
1,76067,AIM076067,796,NaN,NaN,NaN,NaN
2,3488,AIM003488,33,Bacillota,Staphylococcus,Staphylococcus succinus,StaphylococcusStaphylococcus succinus
3,76050,AIM076050,796,NaN,NaN,NaN,NaN
4,76160,AIM076160,797,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
8930,78937,AIM078937,827,Bacillota,Bacillus_A,Bacillus_A thuringiensis_S,Bacillus_ABacillus_A thuringiensis_S
8931,35456,AIM035456,369,Actinomycetota,Curtobacterium,Curtobacterium citreum,CurtobacteriumCurtobacterium citreum
8932,3510,AIM003510,33,Actinomycetota,Glutamicibacter,Glutamicibacter nicotianae,GlutamicibacterGlutamicibacter nicotianae
8933,3522,AIM003522,33,Pseudomonadota,Klebsiella,Klebsiella variicola,KlebsiellaKlebsiella variicola
